In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
diabetes = pd.read_csv('diabetes.csv')
df = diabetes.copy()
df = df.dropna()
y = df['Outcome']
X = df.drop(['Outcome'],axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 44)

In [6]:
scaler = StandardScaler()

In [7]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
mlpc = MLPClassifier()
mlpc_model = mlpc.fit(X_train_scaled, y_train)

In [9]:
#TAHMİN

In [10]:
y_pred = mlpc_model.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.8051948051948052

In [11]:
#TUNING

In [12]:
mlpc_params = {
    'alpha':[0.1, 0.01, 0.02, 0.005, 0.0001, 0.00001],
    'hidden_layer_sizes':[(10,10,10), (100,100,100), (100,100), (3,5), (5,3)],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'activation': ['relu', 'logistic'],
}

In [13]:
mlpc = MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc, mlpc_params, cv = 10, n_jobs = -1, verbose = 2)
mlpc_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  5.1min finished


GridSearchCV(cv=10, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005, 0.0001, 1e-05],
                         'hidden_layer_sizes': [(10, 10, 10), (100, 100, 100),
                                                (100, 100), (3, 5), (5, 3)],
                         'solver': ['adam', 'lbfgs', 'sgd']},
             verbose=2)

In [14]:
mlpc_cv_model.best_params_

{'activation': 'relu',
 'alpha': 0.01,
 'hidden_layer_sizes': (10, 10, 10),
 'solver': 'lbfgs'}

In [23]:
mlpc_tuned = MLPClassifier(activation = 'relu', alpha = 0.01, hidden_layer_sizes = (10, 10, 10), solver = 'lbfgs').fit(X_train_scaled, y_train)

In [24]:
y_pred = mlpc_tuned.predict(X_test_scaled)

In [25]:
accuracy_score(y_test, y_pred)

0.7186147186147186